# Task 3

1. Provide a design strategy for the delivery of the required amount of water from the river to the filtration unit.
2. Evaluate the **velocity** of the water through both pipes for each of the selected *diameters*.
3. Evaluate the **pump** head and **power** required for the *average daily flow* and the *peak flow* including *major* and *minor losses* using the selected pair of *diameters*.
4. Plot the **hydraulic grade line (HGL)** and the **energy line (EL)** along the *length* of the pipe at the *13 locations* for the chosen set of *diameters*.
5. Choose **pump type**, evaluate the **number of pumps** needed, and calulate the **capital cost** of the pumps.
6. Calculate the **capital cost** and the **operating cost** for each pair of *diameters*, evaluate wich of the pairs is **optimum**.

In [1]:
import numpy as np
import fluids
import pandas as pd
import matplotlib.pyplot as plt
from scipy.constants import g

Given Parameters


| Location | Distance (m) | Elevation (m) | Type of fittings | Material | Note |
|---|---|---|---|---|---|
|  0 |  0 | 630 | 2 Elbows, 1 Gate Valve | HDPE | Source
|  1 | 10 | 637 | 1 Reducer, 1 Expander | HDPE
|  2 | 13 | 638 | 4 Elbows | Carbon Steel
|  3 | 15 | 639 || Carbon Steel
|  4 | 19 | 640 | 4 Elbows | Carbon Steel
|  5 | 22 | 641 || Carbon Steel
|  6 | 26 | 642 | 2 Elbows | Carbon Steel
|  7 | 32 | 643 || Carbon Steel
|  8 | 35 | 644 || Carbon Steel
|  9 | 39 | 645 || Carbon Steel
| 10 | 42 | 646 | 4 Elbows | HDPE
| 11 | 45 | 650 | 1 Reducer | HDPE | WTP Entry
| 12 | 45 to 55 | 651 | 8 Elbows, 3 Gate Valves | HDPE | LF
| 13 | 55 to 67 | 657 | 8 Elbows, 2 Gate Valves | HDPE | MF

- Average daily flow rate : 60 L/s
- Peak  flow rate: 180 L/s
- LF pipe length: 10 m
- MF pipe length: 12 m

In [2]:
# Given Data in base SI units
K_L = {'Elbow': 0.39, 'Gate Valve': 0.1, 'Reducer': 0.51, 'Expander': 0.62}

# Surface roughness in m
epsilon = {'HDPE': 1.5e-05, 'Carbon Steel': 1.5e-04}

# Information about pipeline
pipeline = pd.DataFrame(
    {
        'Distance (m)': np.array([0, 10, 13, 15, 19, 22, 26, 32, 35, 39, 42, 45, 55, 67]),
        'Elevation (m)': np.array([630, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 650, 651, 658]),
        'Sum of Loss Coefficients': np.array([
            2*K_L['Elbow']+K_L['Gate Valve'], 
            K_L['Reducer']+K_L['Expander'],
            4*K_L['Elbow'],
            0,
            4*K_L['Elbow'],
            0,
            2*K_L['Elbow'],
            0,
            0,
            0,
            4*K_L['Elbow'],
            K_L['Reducer'],
            8*K_L['Elbow']+3*K_L['Gate Valve'],
            8*K_L['Elbow']+2*K_L['Gate Valve']
            ]),
        'Surface Roughness (m)': np.array([epsilon['HDPE'] for _ in range(2)]
                                        + [epsilon['Carbon Steel'] for _ in range(8)]
                                        + [epsilon['HDPE'] for _ in range(4)]),
        'Section': np.array(['RW' for _ in range(12)]+['WTP' for _ in range(2)])
    }
)

pipeline['Length (m)'] = np.array([0]+[pipeline.iloc[i, 0] - pipeline.iloc[i-1, 0] for i in range(1, len(pipeline.iloc[:,0]))])


# Flow rates in m^3/s
avg_flow_rate = 60/1000
peak_flow_rate = 180/1000

def velocity(flow_rate, diameter):
    '''
    flow_rate : float
        Flow rate in m^3/s
    diameter : float
        Diameter of pipe in meters
    '''
    return 4 * flow_rate / (np.pi * diameter**2)

def Re(velocity, diameter):
    '''
    velocity : float
        Velocity in m/s
    diameter : float
        Diameter of pipe in meters
    '''
    return fluids.Reynolds(velocity, diameter, 1000, 1.787e-03)

def pump_head(z1, v1, z2, v2, h_l):
    '''
    Assuming the initial and final pressures are 1 atm.
    '''
    return (z2-z1) + (v2**2 - v1**2) / (2*g) + h_l

In [3]:
pipeline

,Distance (m),Elevation (m),Sum of Loss Coefficients,Surface Roughness (m),Section,Length (m)
0,0,630,0.88,0.000015,RW,0
1,10,637,1.13,0.000015,RW,10
2,13,638,1.56,0.000150,RW,3
3,15,639,0.00,0.000150,RW,2
4,19,640,1.56,0.000150,RW,4
5,22,641,0.00,0.000150,RW,3
6,26,642,0.78,0.000150,RW,4
7,32,643,0.00,0.000150,RW,6
8,35,644,0.00,0.000150,RW,3
9,39,645,0.00,0.000150,RW,4


In [4]:
average = pd.DataFrame({'D_RW (m)': np.arange(350, 476, 25)/1000, 'D_WTP (m)':np.arange(150, 276, 25)/1000})

peak = average.copy()

average['v_RW (m/s)'] = velocity(avg_flow_rate, average['D_RW (m)'])
peak['v_RW (m/s)'] = velocity(peak_flow_rate, peak['D_RW (m)'])

average['v_WTP (m/s)'] = velocity(avg_flow_rate, average['D_WTP (m)'])
peak['v_WTP (m/s)'] = velocity(peak_flow_rate, peak['D_WTP (m)'])

average['Re_RW'] = Re(average['v_RW (m/s)'], average['D_RW (m)'])
peak['Re_RW'] = Re(peak['v_RW (m/s)'], peak['D_RW (m)'])

average['Re_WTP'] = Re(average['v_WTP (m/s)'], average['D_WTP (m)'])
peak['Re_WTP'] = Re(peak['v_WTP (m/s)'], peak['D_WTP (m)'])

Bernoulli Equation with losses

$$ \frac{P_0}{\rho g} + \frac{v_{0}^2}{2g} + z_0 + h_{pump} = \frac{P_{13}}{\rho g} + \frac{v_{13}^2}{2g} + z_{13} + h_L $$

Losses

$$ h_{L,total} = h_{L,major} + h_{L,minor} = \sum_{i} f_i \frac{L_i}{D_i} \frac{v_i^2}{2g} + \sum_{j} K_{L,j} \frac{v_j^2}{2g} $$

In [8]:
for flow_rate in (average, peak):
    for i in range(len(pipeline)):

        h_loss = []

        for j in range(len(average)):
        
            major = (
                fluids.Churchill_1977(
                flow_rate['Re_{}'.format(pipeline['Section'][i])][j],
                pipeline['Surface Roughness (m)'][i]/flow_rate['D_{} (m)'.format(pipeline['Section'][i])][j])
                * pipeline['Length (m)'][i] * flow_rate['v_{} (m/s)'.format(pipeline['Section'][i])][j]**2
                / (flow_rate['D_{} (m)'.format(pipeline['Section'][i])][j] * 2*g)
            )

            minor = (
                pipeline['Sum of Loss Coefficients'][i]
                * flow_rate['v_{} (m/s)'.format(pipeline['Section'][i])][j]**2
                / (2*g) 
            )

            h_loss.append(major+minor)
        
        flow_rate['Location {} h_L (m)'.format(i)] = h_loss

    flow_rate['Total h_L (m)'] = sum([flow_rate['Location {} h_L (m)'.format(k)] for k in range(14)])

    flow_rate['h_pump (m)'] = pump_head(pipeline['Elevation (m)'][0], flow_rate['v_RW (m/s)'],
                                    pipeline['Elevation (m)'][13], flow_rate['v_WTP (m/s)'],
                                    flow_rate['Total h_L (m)'])


average['Water hp (hp)'] = 1000 * g * avg_flow_rate * average['h_pump (m)'] / 745.7
peak['Water hp (hp)'] = 1000 * g * peak_flow_rate * peak['h_pump (m)'] / 745.7

In [9]:
peak

,D_RW (m),D_WTP (m),v_RW (m/s),v_WTP (m/s),Re_RW,Re_WTP,Location 0 h_L (m),Location 1 h_L (m),Location 2 h_L (m),Location 3 h_L (m),...,Location 7 h_L (m),Location 8 h_L (m),Location 9 h_L (m),Location 10 h_L (m),Location 11 h_L (m),Location 12 h_L (m),Location 13 h_L (m),Total h_L (m),h_pump (m),Water hp (hp)
0,0.350,0.150,1.870883,10.185916,366429.159889,855001.373073,0.157045,0.275084,0.305390,0.017995,...,0.053984,0.026992,0.035989,0.300425,0.113042,22.920713,23.357554,48.080781,81.192246,192.195666
1,0.375,0.175,1.629747,7.483530,342000.549229,732858.319777,0.119171,0.205476,0.230270,0.012674,...,0.038023,0.019011,0.025349,0.226993,0.084800,12.000275,12.161720,25.510360,56.230308,133.106571
2,0.400,0.200,1.432394,5.729578,320625.514903,641251.029805,0.092057,0.156516,0.176900,0.009139,...,0.027417,0.013708,0.018278,0.174684,0.064843,6.874835,6.937570,14.841001,44.410156,105.126291
3,0.425,0.225,1.268834,4.527074,301765.190496,570000.915382,0.072234,0.121282,0.138141,0.006727,...,0.020180,0.010090,0.013454,0.136609,0.050421,4.215844,4.239792,9.253847,38.216687,90.465309
4,0.450,0.250,1.131768,3.666930,285000.457691,513000.823844,0.057471,0.095409,0.109443,0.005042,...,0.015127,0.007563,0.010084,0.108363,0.039790,2.726590,2.734418,6.089853,34.710120,82.164677
5,0.475,0.275,1.015770,3.030521,270000.433602,466364.385313,0.046294,0.076070,0.087828,0.003841,...,0.011524,0.005762,0.007683,0.087054,0.031817,1.840463,1.841442,4.184003,32.599653,77.168848


In [10]:
average

,D_RW (m),D_WTP (m),v_RW (m/s),v_WTP (m/s),Re_RW,Re_WTP,Location 0 h_L (m),Location 1 h_L (m),Location 2 h_L (m),Location 3 h_L (m),...,Location 7 h_L (m),Location 8 h_L (m),Location 9 h_L (m),Location 10 h_L (m),Location 11 h_L (m),Location 12 h_L (m),Location 13 h_L (m),Total h_L (m),h_pump (m),Water hp (hp)
0,0.350,0.150,0.623628,3.395305,122143.053296,285000.457691,0.017449,0.032287,0.034263,0.002220,...,0.006659,0.003329,0.004439,0.033897,0.013077,2.619619,2.682731,5.508579,34.076519,26.888279
1,0.375,0.175,0.543249,2.494510,114000.183076,244286.106592,0.013241,0.024087,0.025836,0.001575,...,0.004725,0.002362,0.003150,0.025598,0.009799,1.371099,1.396585,2.921930,31.224146,24.637598
2,0.400,0.200,0.477465,1.909859,106875.171634,213750.343268,0.010229,0.018326,0.019848,0.001144,...,0.003431,0.001715,0.002287,0.019690,0.007485,0.785000,0.796196,1.698839,29.873189,23.571617
3,0.425,0.225,0.422945,1.509025,100588.396832,190000.305127,0.008026,0.014183,0.015499,0.000847,...,0.002542,0.001271,0.001695,0.015391,0.005815,0.481010,0.486187,1.058468,29.165450,23.013171
4,0.450,0.250,0.377256,1.222310,95000.152564,171000.274615,0.006386,0.011145,0.012279,0.000639,...,0.001917,0.000959,0.001278,0.012204,0.004584,0.310831,0.313276,0.695991,28.764910,22.697123
5,0.475,0.275,0.338590,1.010174,90000.144534,155454.795104,0.005144,0.008876,0.009853,0.000490,...,0.001469,0.000735,0.000979,0.009800,0.003662,0.209634,0.210770,0.477783,28.523966,22.507004
